Draw some balls in a box and have them move around in a random direction and bounce off the walls of the box.

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from pylab import * # to use subplot()
import imageio # to make a gif
from matplotlib.patches import Circle # to draw circles
from matplotlib.collections import PatchCollection # used in circles

The scatter from pyplot doesn't let you control the width of the points in a sensible way. So, we'll use the following circles function. We need this to compute when the boundary of the ball hits the walls.

In [13]:
def circles(x, y, s, c='b', vmin=None, vmax=None, **kwargs):
    """
    source: https://gist.github.com/syrte/592a062c562cd2a98a83
    Make a scatter plot of circles. 
    Similar to plt.scatter, but the size of circles are in data scale.
    Parameters
    ----------
    x, y : scalar or array_like, shape (n, )
        Input data
    s : scalar or array_like, shape (n, ) 
        Radius of circles.
    c : color or sequence of color, optional, default : 'b'
        `c` can be a single color format string, or a sequence of color
        specifications of length `N`, or a sequence of `N` numbers to be
        mapped to colors using the `cmap` and `norm` specified via kwargs.
        Note that `c` should not be a single numeric RGB or RGBA sequence 
        because that is indistinguishable from an array of values
        to be colormapped. (If you insist, use `color` instead.)  
        `c` can be a 2-D array in which the rows are RGB or RGBA, however. 
    vmin, vmax : scalar, optional, default: None
        `vmin` and `vmax` are used in conjunction with `norm` to normalize
        luminance data.  If either are `None`, the min and max of the
        color array is used.
    kwargs : `~matplotlib.collections.Collection` properties
        Eg. alpha, edgecolor(ec), facecolor(fc), linewidth(lw), linestyle(ls), 
        norm, cmap, transform, etc.
    Returns
    -------
    paths : `~matplotlib.collections.PathCollection`
    Examples
    --------
    a = np.arange(11)
    circles(a, a, s=a*0.2, c=a, alpha=0.5, ec='none')
    plt.colorbar()
    License
    --------
    This code is under [The BSD 3-Clause License]
    (http://opensource.org/licenses/BSD-3-Clause)
    """

    if np.isscalar(c):
        kwargs.setdefault('color', c)
        c = None

    if 'fc' in kwargs:
        kwargs.setdefault('facecolor', kwargs.pop('fc'))
    if 'ec' in kwargs:
        kwargs.setdefault('edgecolor', kwargs.pop('ec'))
    if 'ls' in kwargs:
        kwargs.setdefault('linestyle', kwargs.pop('ls'))
    if 'lw' in kwargs:
        kwargs.setdefault('linewidth', kwargs.pop('lw'))
    # You can set `facecolor` with an array for each patch,
    # while you can only set `facecolors` with a value for all.

    zipped = np.broadcast(x, y, s)
    patches = [Circle((x_, y_), s_)
               for x_, y_, s_ in zipped]
    collection = PatchCollection(patches, **kwargs)
    if c is not None:
        c = np.broadcast_to(c, zipped.shape).ravel()
        collection.set_array(c)
        collection.set_clim(vmin, vmax)

    ax = plt.gca()
    ax.add_collection(collection)
    ax.autoscale_view()
    plt.draw_if_interactive()
    if c is not None:
        plt.sci(collection)
    return collection

Here is when the computations and drawing happens. 

To do:
- take into account the direction of the balls when measuring when it hits the wall so that they don't bounce off too early or too late.

In [10]:
# define some parameters
steps = 400 # number of steps that the walkers take
walkers = 100 # number of random walkers

lim = 10 # define plot limits
length = lim/20 #* (1 + np.random.rand(walkers)) # pick step length 
size = .5 # radius of dots
margin = 2*size #/ (np.sqrt(96)) # margin for when to reflect from the boundary

# randomize the initial position of each walker
concentration = lim - 2 * margin # set equal to zero to see a completely different behavior
xi = 2*np.random.rand(walkers)*concentration - concentration
yi = 2*np.random.rand(walkers)*concentration - concentration

# define colors - give each walker its own colors
colors = np.random.rand(walkers) # colors of the points
#colors = plt.cm.gist_ncar(np.linspace(0,1,walkers))

# angle for each walker
angle = np.random.rand(walkers)*2*np.pi

#margined = []

# go through steps
for step in range(1,steps):
    
    ax=subplot(aspect='equal')
    #circles(xi[margined], yi[margined], 3*size, c='w', alpha=.1, edgecolor='none')
    
    # update position
    xi = xi+np.cos(angle)*length
    yi = yi+np.sin(angle)*length
    
    # reflect at boundaries
    angle[xi+margin > lim] = np.pi - angle[xi+margin > lim]
    angle[xi-margin < -lim] = np.pi - angle[xi-margin < -lim]
    angle[yi+margin > lim] = - angle[yi+margin > lim]
    angle[yi-margin < -lim] = - angle[yi-margin < -lim]
    
    #margined = [np.array(xi+margin > lim) | np.array(xi-margin < -lim) | np.array(yi+margin > lim) | np.array(yi-margin < -lim)]
    #circles(xi[margined], yi[margined], 2*size, c='w', alpha=.2, edgecolor='none')
    
    # plot the walkers 
    circles(xi, yi, size, c=colors, alpha=.6, edgecolor='none',cmap=plt.get_cmap('tab20b'))
    
    # plot the boundary
    plt.plot([-lim,lim,lim,-lim,-lim], [-lim,-lim,lim,lim,-lim], 'w-')
    
    # set x and y limits
    xlim(-lim,lim)
    ylim(-lim,lim)

    plt.axis('off')
    plt.savefig('%s_particles_%04d.png' %(walkers,step),dpi=96,facecolor='black',bbox_inches='tight',pad=-1)
    plt.close()

make a gif file from the png files created

In [12]:
# make a gif file 
with imageio.get_writer('movie4.gif', mode='I', duration=0) as writer:
    for step in range(1,steps,1):
        filename = '%s_particles_%04d.png' %(walkers,step)
        image = imageio.imread(filename)
        writer.append_data(image)